### Self Implementation of DefaultModelMonitor

#### Step 1 : Define a simple regression use case

#### Step 2 : Create Code to input data, generate inferences using batch transform

##### Step 2.1 : Get a very simple dataset of age and salary with some randomness 

1. See data folder. 

##### Step 2.2 : Create a code to generate inference using batch transform. 

##### Step 2.3 : Provide a sample batch input file and generate inferences

#### Step 3 : Create sagemaker pipeline to train model and generate inference again when a new train input file is loaded

##### Step 3.1 : Create a sample pipeline on existing train data

##### Step 3.2 : Execute pipeline and generate inference on existing train data

##### Step 3.3 : Upload new train data and generate inference by executing pipeline

#### Step 4 : DefaultModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints

##### Step 4.1 : Code for monitor data quality config and job using DefaultModelMonitor

##### Step 4.2 : Get a batch input file with outlier that the monitor data quality step should REJECT

##### Step 4.3 : Get a batch input file with outlier that the monitor data quality step should ACCEPT

#### Step 5 : ModelMonitor : Create code to monitor data quality and accept/reject batch input file based on constraints

##### Step 5.1 : Code for monitor data quality config and job using ModelMonitor

##### Step 5.2 : Get a batch input file with outlier that the monitor data quality step should REJECT

##### Step 5.3 : Get a batch input file with outlier that the monitor data quality step should ACCEPT

#### Step X : End Of Notebook